In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.utils import np_utils
from tflearn.data_utils import to_categorical
from sklearn.model_selection import train_test_split

In [ ]:
def getRawFeatures(picture):
    red = []
    green = []
    blue = []
    for row in range(picture.shape[0]):
        for col in range(picture.shape[1]):
            red.append(picture[row][col][0])
            green.append(picture[row][col][1])
            blue.append(picture[row][col][2])
    feature = red
    feature.extend(green)
    feature.extend(blue)
    return feature

In [ ]:
import io
from tqdm import tqdm_notebook
import bson # this is installed with the pymongo package
import matplotlib
import matplotlib.pyplot as plt
from skimage.data import imread   # or, whatever image library you prefer
import multiprocessing as mp      # will come in handy due to the size of the data
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
# # The images are being loaded :)
# # It will take a while, as the total size is too large

# # Simple data processing
# count_images = 0
# image_names_array = []
# category_id_array = []
# data = bson.decode_file_iter(open('../input/train.bson', 'rb'))
# pictures = []
# count = 0
# prod_to_category = dict()

# for c, d in enumerate(data):
#     #for each product_id
#     product_id = d['_id']
#     category_id = d['category_id'] # This won't be in Test data
#     prod_to_category[product_id] = category_id
    
#     for e, pic in enumerate(d['imgs']):
#         #for each image
#         picture = imread(io.BytesIO(pic['picture']))
#         pictures.append(picture)
#         count = count + 1
#         # do something with the picture, etc
# #         image_name = "prod_id-" + str(product_id) + "-" + "image-" + str(e)
# #         print("PRODUCT ID:", product_id, "NUMBER", e)
# #         plt.imshow(picture)
# #         fig1 = plt.gcf()
# #         plt.show()
# #         plt.draw()
#         count_images = count_images + 1
# #         image_names_array.append(image_name)
#         category_id_array.append(str(category_id))
#         #fig1.savefig("img/" + str(image_name), dpi=100)
#     print(count_images)
# #     break

# prod_to_category = pd.DataFrame.from_dict(prod_to_category, orient='index')
# prod_to_category.index.name = '_id'
# prod_to_category.rename(columns={0: 'category_id'}, inplace=True)

In [ ]:
pictures = []
category_id_array = []
prod_id = []
prod_category = []
prod_num_imgs = []

num_dicts = 82 # according to data page

count = 0
# This will take about 02m15s to complete
with open('../input/train_example.bson', 'rb') as f, tqdm_notebook(total=num_dicts) as bar:
    
    data = bson.decode_file_iter(f)

    for c, d in enumerate(data):
        bar.update()
        count = count + 1
#         if(count > 82):
#             break
        prod_id.append(d['_id'])
        prod_category.append(str(d['category_id']))
        prod_num_imgs.append(len(d['imgs']))
        
        for e, pic in enumerate(d['imgs']):
            #for each image
            picture = imread(io.BytesIO(pic['picture']))
            pictures.append(picture)
            category_id_array.append(str(d['category_id']))

In [ ]:
#loading a subset of data,  as it is tooo largeee
from collections import Counter
X = np.asarray(pictures)
y = np.asarray(category_id_array)
y.shape
numClasses = len(Counter(y))

In [ ]:
#X has the RGB values of images, y has the category labels
print(X.shape)
print(y.shape)

In [ ]:
# X = X.reshape(X.shape[0], 3, 180, 180).astype('float32')
# X = X - np.mean(X) / X.std()

In [ ]:
# y_train

In [ ]:
# b,c = np.unique(y_train, return_inverse=True)

In [ ]:
# from collections import Counter
# # d = Counter(c)

In [ ]:
# y_train = c

In [ ]:
# y_train = np_utils.to_categorical(y_train)

In [ ]:
# X_train = X_train.reshape(X_train.shape[0], 180, 180, 3)
# X_train = X_train.astype('float32')
# X_train = X_train/255
print(X.shape, y.shape)

In [ ]:
X = X.reshape(X.shape[0], 180, 180, 3)
X = X.astype('float32')
X/=255
b, y = np.unique(y, return_inverse=True)
y = np_utils.to_categorical(y)
print(y)

#lets split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
X_train = X
y_train = y

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(180,180,3)))
BatchNormalization(axis=-1)
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
BatchNormalization(axis=-1)
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3, 3)))
BatchNormalization(axis=-1)
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
BatchNormalization(axis=-1)
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

# Fully connected layer
model.add(Dense(512))

BatchNormalization()
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(numClasses))

model.add(Activation('softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [ ]:
#this is the total number of classes in the chosen subset of the actual data
numClasses

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
gen = ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3,
                         height_shift_range=0.08, zoom_range=0.08)
test_gen = ImageDataGenerator()

In [ ]:
#the generator functions are created so as to, pass the train and test sets to the learner function
#helps us keeping our hands clean
train_generator = gen.flow(X_train, y_train, batch_size=10)
test_generator = test_gen.flow(X_test, y_test, batch_size=10)

numTrainExamples = len(X_train)
numTrainExamples
numTestExamples = len(X_test)
batchSize = 10#int((1/13)*numTrainExamples)
numEpochs = 10

#Even for this dataset this is going to take a while
#After doing on this, we did on a larger subset, but kaggle kernal didn't allow us to do so :/
#The kernal died doing this enormous amount of processing on RGB images

In [ ]:
model.fit_generator(train_generator, steps_per_epoch=numTrainExamples//batchSize, epochs=numEpochs, validation_data=test_generator, validation_steps=numTestExamples//batchSize)

In [ ]:
y_predict = model.predict(X_test)

In [ ]:
len(y_predict)
# # y_predict.index(max(y_predict[0]))
indices_predict = np.argmax(y_predict, axis=1)
c = Counter(indices_predict)
print(c)
# indices_predict
indices_test = np.argmax(y_test, axis=1)
# print(indices_predict)
# print(indices_test)



In [ ]:
# print(arr[0].index(max(arr[0])))

In [ ]:
# arr[0]

In [ ]:
# type(indices_test)

In [ ]:
# from sklearn.metrics import roc_auc_score, roc_curve
# from sklearn import metrics

# # roc_auc_score(indices_test, indices_predict)
indices_predict = np.array(indices_predict.tolist())
indices_test = np.array(indices_test.tolist())
# metrics.roc_curve(indices_test, indices_predict)
print(indices_predict, indices_test)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp

# # Import some data to play with
# iris = datasets.load_iris()
# X = iris.data
# y = iris.target

# Binarize the output
# y = label_binarize(y, classes=[0, 1, 2])
n_classes = numClasses

# # Add noisy features to make the problem harder
# random_state = np.random.RandomState(0)
# n_samples, n_features = X.shape
# X = np.c_[X, random_state.randn(n_samples, 200 * n_features)]

# # shuffle and split training and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.5,
#                                                     random_state=0)

# # Learn to predict each class against the other
# classifier = OneVsRestClassifier(svm.SVC(kernel='linear', probability=True,
#                                  random_state=random_state))
# y_score = classifier.fit(X_train, y_train).decision_function(X_test)

y_score = y_predict
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

In [ ]:
plt.figure()
lw = 2
plt.plot(fpr[2], tpr[2], color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[2])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# Compute macro-average ROC curve and ROC area

# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure()
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
#plt.legend(loc="lower right")
plt.show()

In [ ]:
y_score

In [ ]:
y_test